In [1]:
import pandas as pd
import numpy as np


data files can be found here: https://drive.google.com/drive/folders/1TPlfyhG1JxjaG67tzR9G4uzhaRnI6UVb

In [2]:
kiet_labels = '/Users/trnan/OneDrive/Desktop/uw/Tutill lab/repos/downstream_hemilineage_labels_063122_KT.csv'
kiet_df = pd.read_csv(kiet_labels)

leila_labels = '/Users/trnan/OneDrive/Desktop/uw/Tutill lab/repos/downstream_hemilineage_labels_063122_LE.csv'
leila_df = pd.read_csv(leila_labels)

kiet_df = kiet_df[['Coordinate 1', 'Description', 'Segment IDs']]
leila_df = leila_df[['Coordinate 1', 'Description', 'Segment IDs']]

kiet_df.head()

,Coordinate 1,Description,Segment IDs
0,"(17920, 116312, 1540)",23B,648518346502418246
1,"(12600, 116296, 2413)",3A,648518346517407524
2,"(20200, 102904, 3801)",Error,648518346496076044
3,"(12728, 115580, 2467)",3A Unsure,648518346506602274
4,"(17912, 115196, 1541)",23B,648518346492755118


In [3]:
leila_df.head()

,Coordinate 1,Description,Segment IDs
0,"(17920, 116312, 1540)",23B,648518346502418246
1,"(12600, 116296, 2413)",3A,648518346517407524
2,"(20200, 102904, 3801)",Error,648518346496076044
3,"(12728, 115580, 2467)",3A Unsure,648518346506602274
4,"(17912, 115196, 1541)",23B,648518346492755118


In [4]:
kiet_df['Segment IDs'].tolist() == leila_df['Segment IDs'].tolist()
kietDesc = kiet_df['Description'].tolist()
leilaDesc = leila_df['Description'].tolist()

leila_df['Description'] = leila_df['Description'].str.lower()
kiet_df['Description'] = kiet_df['Description'].str.lower()
kiet_df['Description']

0             23b
1              3a
2           error
3       3a unsure
4             23b
          ...    
140        unsure
141    20a unsure
142            7b
143     9a unsure
144        unsure
Name: Description, Length: 145, dtype: object

### To Do

- merge the tables on Segment ID (add a suffix to description column with le or kt)
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

- compare the two description columns:
- add column to the table with labels for agreements 
- add column for whether there is an unsure (can be done in one line of code)


In [5]:
merged = kiet_df.merge(leila_df, on=['Coordinate 1','Segment IDs'],suffixes=('_kt', '_le'))
merged.tail()

,Coordinate 1,Description_kt,Segment IDs,Description_le
140,"(59296, 191320, 367)",unsure,648518346498343152,unsure
141,"(58800, 187584, 286)",20a unsure,648518346483056204,unsure
142,"(52776, 172056, 2657)",7b,648518346500584371,unsure
143,"(34792, 177664, 2909)",9a unsure,648518346500304700,unsure
144,"(61488, 196368, 555)",unsure,648518346478593140,unsure


In [6]:
comp = kiet_df['Description'].compare(leila_df['Description'], keep_shape=True)
inAgreement = comp.notna()
print(inAgreement)
inAgreement = inAgreement.replace(False, 'same')
inAgreement = inAgreement.replace(True, 'different')
inAgreement.rename(columns = {'self':'In Agreement'}, inplace=True)
merged = pd.concat([merged,inAgreement['In Agreement']], axis=1);
merged

      self  other
0    False  False
1    False  False
2    False  False
3    False  False
4    False  False
..     ...    ...
140  False  False
141   True   True
142   True   True
143   True   True
144  False  False

[145 rows x 2 columns]


,Coordinate 1,Description_kt,Segment IDs,Description_le,In Agreement
0,"(17920, 116312, 1540)",23b,648518346502418246,23b,same
1,"(12600, 116296, 2413)",3a,648518346517407524,3a,same
2,"(20200, 102904, 3801)",error,648518346496076044,error,same
3,"(12728, 115580, 2467)",3a unsure,648518346506602274,3a unsure,same
4,"(17912, 115196, 1541)",23b,648518346492755118,23b,same
...,...,...,...,...,...
140,"(59296, 191320, 367)",unsure,648518346498343152,unsure,same
141,"(58800, 187584, 286)",20a unsure,648518346483056204,unsure,different
142,"(52776, 172056, 2657)",7b,648518346500584371,unsure,different
143,"(34792, 177664, 2909)",9a unsure,648518346500304700,unsure,different


In [7]:
unsureCheck = merged[['Description_kt','Description_le']]
substrCheck = unsureCheck['Description_kt'].str.find('unsure') != -1
substrCheck2 = unsureCheck['Description_le'].str.find('unsure') != -1
unsureCheck = unsureCheck.mask(substrCheck, 'unsure')
print(unsureCheck)
unsureCheck = unsureCheck.mask(substrCheck2, 'unsure')
print(unsureCheck)
unsureCheck = unsureCheck.mask(unsureCheck != 'unsure', 'ok')
unsureCheck.rename(columns={'Description_kt':'Unsure Check'}, inplace=True)
merged = pd.concat([merged,unsureCheck['Unsure Check']], axis=1);
merged

    Description_kt Description_le
0              23b            23b
1               3a             3a
2            error          error
3           unsure         unsure
4              23b            23b
..             ...            ...
140         unsure         unsure
141         unsure         unsure
142             7b         unsure
143         unsure         unsure
144         unsure         unsure

[145 rows x 2 columns]
    Description_kt Description_le
0              23b            23b
1               3a             3a
2            error          error
3           unsure         unsure
4              23b            23b
..             ...            ...
140         unsure         unsure
141         unsure         unsure
142         unsure         unsure
143         unsure         unsure
144         unsure         unsure

[145 rows x 2 columns]


,Coordinate 1,Description_kt,Segment IDs,Description_le,In Agreement,Unsure Check
0,"(17920, 116312, 1540)",23b,648518346502418246,23b,same,ok
1,"(12600, 116296, 2413)",3a,648518346517407524,3a,same,ok
2,"(20200, 102904, 3801)",error,648518346496076044,error,same,ok
3,"(12728, 115580, 2467)",3a unsure,648518346506602274,3a unsure,same,unsure
4,"(17912, 115196, 1541)",23b,648518346492755118,23b,same,ok
...,...,...,...,...,...,...
140,"(59296, 191320, 367)",unsure,648518346498343152,unsure,same,unsure
141,"(58800, 187584, 286)",20a unsure,648518346483056204,unsure,different,unsure
142,"(52776, 172056, 2657)",7b,648518346500584371,unsure,different,unsure
143,"(34792, 177664, 2909)",9a unsure,648518346500304700,unsure,different,unsure


In [8]:
merged.to_pickle("C:/Users/trnan/OneDrive/Desktop/uw/Tutill lab/repos/merged.pkl")